<center><h1><font color=yellow><i> Ozone - Random Foorest </i></font></h1></center>

In [140]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Data

Import Data

In [141]:
data=pd.read_csv('ozone_complet_preprocessed.csv', sep=',')
data.head()

,Unnamed: 0,date,maxO3,T6,T9,T12,T15,T18,Ne6,Ne9,...,Vvit9,Vdir12,Vvit12,Vdir15,Vvit15,Vdir18,Vvit18,Vx,maxO3v,maxO3l
0,0,1995-04-01,47.6,10.1,11.6,13.3,13.6,12.2,8.0,8.0,...,4.0,300.0,4.0,340.0,4.0,20.0,4.0,-3.4641,62.2,56.2
1,1,1995-04-02,56.2,9.5,9.4,13.8,17.4,16.3,8.0,8.0,...,2.0,180.0,3.0,110.0,1.0,350.0,2.0,0.0000,47.6,61.8
2,2,1995-04-03,61.8,3.6,8.0,16.8,21.5,20.2,4.0,5.0,...,2.0,340.0,1.0,170.0,2.0,170.0,3.0,-0.3420,56.2,50.8
3,3,1995-04-04,50.8,9.5,10.5,11.4,12.2,11.4,8.0,7.0,...,4.0,350.0,3.0,350.0,3.0,350.0,4.0,-0.5209,61.8,59.8
4,4,1995-04-05,59.8,9.8,10.8,13.8,14.3,13.3,8.0,7.0,...,2.0,280.0,1.0,320.0,3.0,350.0,4.0,-0.9848,50.8,53.4


In [142]:
df_ozone=data

In [143]:

# Supposons que 'df' est votre DataFrame contenant la colonne 'date'
# Convertir la colonne 'date' en type datetime si elle ne l'est pas déjà
df_ozone['date'] = pd.to_datetime(df_ozone['date'])

# Définir une date de référence
date_reference = pd.to_datetime('1995-04-01')

# Calculer le nombre de jours depuis la date de référence
df_ozone['days_since_reference'] = (df_ozone['date'] - date_reference).dt.days

# Afficher les résultats
print(df_ozone[['date', 'days_since_reference']])
df_ozone=df_ozone.drop(columns='date',axis=1)


           date  days_since_reference
0    1995-04-01                     0
1    1995-04-02                     1
2    1995-04-03                     2
3    1995-04-04                     3
4    1995-04-05                     4
...         ...                   ...
1385 2002-09-25                  2734
1386 2002-09-26                  2735
1387 2002-09-27                  2736
1388 2002-09-28                  2737
1389 2002-09-29                  2738

[1390 rows x 2 columns]


Features and output

In [144]:
df_ozone.drop('Unnamed: 0', axis=1, inplace=True)
y=df_ozone['maxO3l']
X=df_ozone.drop('maxO3l',axis=1)

In [145]:
X.head()

,maxO3,T6,T9,T12,T15,T18,Ne6,Ne9,Ne12,Ne15,...,Vvit9,Vdir12,Vvit12,Vdir15,Vvit15,Vdir18,Vvit18,Vx,maxO3v,days_since_reference
0,47.6,10.1,11.6,13.3,13.6,12.2,8.0,8.0,8.0,8.0,...,4.0,300.0,4.0,340.0,4.0,20.0,4.0,-3.4641,62.2,0
1,56.2,9.5,9.4,13.8,17.4,16.3,8.0,8.0,7.0,0.0,...,2.0,180.0,3.0,110.0,1.0,350.0,2.0,0.0000,47.6,1
2,61.8,3.6,8.0,16.8,21.5,20.2,4.0,5.0,2.0,2.0,...,2.0,340.0,1.0,170.0,2.0,170.0,3.0,-0.3420,56.2,2
3,50.8,9.5,10.5,11.4,12.2,11.4,8.0,7.0,7.0,7.0,...,4.0,350.0,3.0,350.0,3.0,350.0,4.0,-0.5209,61.8,3
4,59.8,9.8,10.8,13.8,14.3,13.3,8.0,7.0,8.0,8.0,...,2.0,280.0,1.0,320.0,3.0,350.0,4.0,-0.9848,50.8,4


In [146]:
X.describe()

,maxO3,T6,T9,T12,T15,T18,Ne6,Ne9,Ne12,Ne15,...,Vvit9,Vdir12,Vvit12,Vdir15,Vvit15,Vdir18,Vvit18,Vx,maxO3v,days_since_reference
count,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,...,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000
mean,84.928269,12.303096,16.388913,19.420806,20.346364,18.783153,4.998559,5.159942,5.337176,5.150576,...,3.732901,205.017999,4.508279,208.351332,4.782577,190.820734,4.392369,-0.747871,85.037059,1384.066906
std,23.142702,3.898741,4.127919,4.926189,5.269710,5.079061,2.908986,2.564151,2.315037,2.323004,...,1.872830,105.871965,2.056291,110.564767,2.063139,121.417902,1.765431,3.401020,23.066851,848.879846
min,33.200000,-1.600000,4.500000,3.900000,6.500000,4.400000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-10.392300,33.200000,0.000000
25%,68.250000,10.000000,13.800000,16.225000,16.800000,15.500000,2.000000,3.000000,4.000000,3.000000,...,2.000000,110.000000,3.000000,100.000000,3.000000,60.000000,3.000000,-3.064200,68.650000,531.250000
50%,81.000000,12.700000,16.700000,19.400000,20.400000,18.600000,6.000000,6.000000,6.000000,6.000000,...,4.000000,220.000000,4.000000,230.000000,5.000000,220.000000,4.000000,-1.026100,81.600000,1481.500000
75%,97.000000,15.100000,19.000000,22.600000,23.800000,22.000000,8.000000,7.000000,7.000000,7.000000,...,5.000000,300.000000,6.000000,300.000000,6.000000,290.000000,6.000000,1.500000,97.000000,2202.750000
max,173.000000,21.700000,28.600000,33.500000,35.500000,34.100000,9.000000,9.000000,8.000000,8.000000,...,12.000000,360.000000,12.000000,360.000000,16.000000,360.000000,12.000000,10.832900,173.000000,2738.000000


Split our data into **train data** (80% -- 3680 rows) and **test data** (921 rows). 

In [147]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [148]:
from sklearn.preprocessing import StandardScaler
# Initialisation du scaler
scaler = StandardScaler()
# Normaliser uniquement les caractéristiques (X)
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Random forest

In [158]:
# Initialisation et entraînement du modèle de forêt aléatoire
model = RandomForestRegressor(max_depth=10,n_estimators=250,min_samples_leaf=4 , min_samples_split=2, random_state=42)
model.fit(X_train_normalized, y_train)

# Prédiction sur l'ensemble de test
predictions = model.predict(X_test_normalized)

# Évaluation du modèle
mse = mean_squared_error(y_test, predictions)
print(f"MSE (Mean Squared Error): {mse}")
#MSE (Mean Squared Error): 268.52521267083216


MSE (Mean Squared Error): 235.97027505620702


In [159]:
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score


# Calcul du RMSE (Root Mean Squared Error)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

# Calcul du MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, predictions)
print(f"MAE: {mae}")

# Coefficient de détermination (R²)
r2 = r2_score(y_test, predictions)
print(f"R² Score: {r2}")


RMSE: 15.361324000756152
MAE: 11.79332390849757
R² Score: 0.5792719579316543


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Définition des hyperparamètres à tester
param_grid = {
    'n_estimators': [100, 200, 300],  # Nombre d'arbres dans la forêt
    'max_depth': [10, 20, 30],  # Profondeur maximale des arbres
    'min_samples_split': [2, 5, 10],  # Nombre minimum d'échantillons requis pour diviser un nœud
    'min_samples_leaf': [ 2, 4]  # Nombre minimum d'échantillons requis pour être une feuille
}

# Initialisation du modèle Random Forest
rf = RandomForestRegressor()

# Recherche des meilleurs hyperparamètres à l'aide de GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Entraînement du modèle avec Grid Search
grid_search.fit(X_train_normalized, y_train)

# Affichage des meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres:")
print(grid_search.best_params_)

# Utilisation du meilleur modèle trouvé pour les prédictions
best_rf = grid_search.best_estimator_
predictions = best_rf.predict(X_test_normalized)


Fitting 5 folds for each of 54 candidates, totalling 270 fits
Meilleurs hyperparamètres:
{'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}
